In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import partial

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torchvision.transforms import ToTensor

from utils.dataset import VideoDataset
from utils.training import train_model
from utils.models import TrackNetV2MSE, TrackNetV2NLL

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

%load_ext autoreload
%autoreload 2

## Dataset

In [ ]:
sequence_length = 3

dataset_params = dict(root="../videos/dataset_finales_2020_en/",
                      image_size=(512, 512),
                      sigma=5,
                      sequence_length=sequence_length,
                      heatmap_mode='image',
                      duplicate_equality_threshold=0.97,
                      one_output_frame=False,
                      drop_duplicate_frames=True)

dataset_demo = VideoDataset(**dataset_params)
dataset_train = VideoDataset(**dataset_params, transform=ToTensor(), target_transform=ToTensor(), split='train')
dataset_val = VideoDataset(**dataset_params, transform=ToTensor(), target_transform=ToTensor(), split='val')

data_loader_train = DataLoader(dataset_train, batch_size=2)
data_loader_val = DataLoader(dataset_val, batch_size=2)

In [ ]:
frames, heatmap = dataset_train[0]

In [ ]:
w, h, dpi = 480, 853, 50

fig, ax = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi)

frames, labels = dataset_demo[66]
ax.imshow(frames[2])
ax.imshow(labels[2], alpha=0.6, cmap='gray')
ax.set_axis_off()
fig.tight_layout(pad=0)

plt.show()

# Training

Load model and set checkpoint folder

In [ ]:
model = TrackNetV2MSE(sequence_length=sequence_length, one_output_frame=False)
checkpoint_folder = './checkpoints/checkpoints_512_mse'

train_model_partial = partial(train_model,
                              loss_function=F.mse_loss,
                              checkpoint_folder=checkpoint_folder)

In [ ]:
model = TrackNetV2NLL(sequence_length=sequence_length)
checkpoint_folder = './checkpoints/checkpoints_512_nll'

train_model_partial = partial(train_model,
                              loss_function=F.nll_loss,
                              checkpoint_folder=checkpoint_folder)

Training loop

In [ ]:
#TODO: check the loss function
#TODO: check the training recipe

checkpoint_dict = train_model_partial(model,
                                      data_loader_train,
                                      data_loader_val,
                                      epochs=1,
                                      device=device)

Plot train and validation loss

In [ ]:
train_loss = np.loadtxt(os.path.join(checkpoint_folder, "loss_history.csv"))
val_loss = np.loadtxt(os.path.join(checkpoint_folder, "loss_history_val.csv"))

fig, ax = plt.subplots()
ax.set_yscale('log')

ax.plot(train_loss, label='train loss')
ax.plot(val_loss, label='val loss')

ax.set_xlabel("Epoch")
ax.set_ylabel("Loss")

ax.legend()

plt.show()

Load weights from checkpoint

In [ ]:
from utils.storage import load_weights

load_weights(model, "checkpoints\checkpoints_512\checkpoint_0101_best.ckpt")
model.to(device);

In [ ]:
model.load("checkpoints\checkpoints_512_mse\checkpoint_0101_best.ckpt", device=device)
model.eval();

# Show example

Get maximum of the heatmap

In [ ]:
def get_maximum_coordinates(heatmaps):
    if len(heatmaps.shape)==3:
        map_max_values = np.max(heatmaps.reshape(3, -1), axis=1)
        map_index = np.argmax(map_max_values)
        max_value = np.max(map_max_values)
        x, y = np.nonzero(heatmaps[map_index] == max_value)
    else:
        max_value = np.max(heatmaps)
        x, y = np.nonzero(heatmaps == max_value)

    return y[0], x[0]

Produce output heatmap

In [ ]:
frames, heatmaps = dataset_val[20]
heatmaps_np = heatmaps.numpy()
frames_np = frames.numpy()[:3].transpose(1, 2, 0)

with torch.no_grad():
    outputs = model(frames[None, :].to(device))
outputs_np = outputs.to('cpu').numpy()[0].transpose(0, 1, 2)

In [ ]:
w, h, dpi = 1280, 720, 150

fig, axs = plt.subplots(figsize=(w/dpi, h/dpi), dpi=dpi, ncols=2)

axs[0].imshow(frames_np)

axs[1].imshow(outputs_np[0], cmap='magma')
axs[1].scatter(*get_maximum_coordinates(heatmaps_np[0]), color='w', label='True position')
axs[1].scatter(*get_maximum_coordinates(outputs_np), color='y', label='Predicted position')

axs[1].legend(framealpha=0.5)

fig.tight_layout()
for ax in axs:
    ax.set_axis_off()

plt.show()

# Duplicate frames demo

In [ ]:
import cv2
root = "../videos/dataset_finales_2020_en/"
cap = cv2.VideoCapture(os.path.join(root, "video.mp4"))

In [ ]:
cap.set(cv2.CAP_PROP_POS_FRAMES, 13620)
_, frame1 = cap.read()
frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)

_, frame2 = cap.read()
frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2RGB)

In [ ]:
plt.imshow(frame1)
plt.show()

plt.imshow(frame2)
plt.show()

In [ ]:
plt.imshow(np.abs(frame2 - frame1))
plt.colorbar()
plt.show()